In [54]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [55]:
#Testing
scrape_link = 'https://www.apartments.com/apartments/ca/'

response = requests.get(scrape_link)
soup = BeautifulSoup(response.text, 'html.parser')
result = soup.find_all('div', class_="placardContainer")
result

[<div class="placardContainer" id="placardContainer">
 <ul>
 <li>
 <article class="diamond placard" data-ck="fgm2gzv" data-listingid="y9fg55t" data-url="https://www.apartments.com/homecoming-at-eastvale-eastvale-ca/y9fg55t/">
 <header class="placardHeader">
 <a class="placardTitle js-placardTitle " href="https://www.apartments.com/homecoming-at-eastvale-eastvale-ca/y9fg55t/" title="Homecoming At Eastvale, Eastvale, CA">
 Homecoming At Eastvale</a>
 <img alt="Lewis Management Corp" class="propertyLogo" src="https://images1.apartments.com/i2/8oJ0IbanGmv-y64BtgyDHrIkQjD07arn9GPAfsdd30g/110/lewis-management-corp-logo.png"/>
 <div class="location" title="5464 W Homecoming Cir, Eastvale, CA 91752">5464 W Homecoming Cir, Eastvale, CA 91752</div>
 </header>
 <section class="placardContent">
 <div class="media">
 <div class="placardCarouselImgCount">
 <span class="js-spnImgNum">1</span> / <span class="js-spnImgCount">25</span>
 </div>
 <div class="galleryControl">
 <button class="imageCarouselL

In [56]:
#Set up list to hold response info
link_dict = []
all_property = []

In [57]:
scrape_link = 'https://www.apartments.com/apartments/ca/{}'
# Loop through x pages of the website with filter of Single House
for scrape_links in [scrape_link.format(page) for page in range(20,25)]:
    res = requests.get(scrape_links)
    new_soup = BeautifulSoup(res.text, 'html.parser')
    new_results = new_soup.find_all('article', class_="placard")
    
    # Loop through returned results
    for result in new_results:
        find_tag = result.find('a')['href']
        link_dict.append(find_tag)

for property_link in link_dict:
    res = requests.get(property_link)
    new_soup = BeautifulSoup(res.text, 'html.parser')
    new_results = new_soup.find_all('div', class_="mainWrapper")
    
    
    for result in new_results:
        try:
            # Identify and return property name of Rental Place
            property_name = result.find('h1', class_="propertyName")
            # Identify and return address information of Rental Place
            clean_property_name = property_name.text.strip().replace("\n", "")
            street_data = result.find('div', class_="propertyAddress")
            street = street_data.text.strip().replace("\n", "").replace("\r", "")
            street_address = street.split(",")
            city = " ".join(street_address[1].split())
            state = " ".join(street_address[2].split())[:2]
            zip_code = " ".join(street_address[2].split())[2:7]
            # Identify and return number of bedroom and price of Rental Place
            bedroom_container = result.find('div', class_="rentRollupContainer")
            all_the_rents = bedroom_container.findAll('span', class_="rentRollup")
            
            bedroom_info = []
            for rent_block in all_the_rents:
                
                bedroom_type = rent_block.find('span', class_="longText").text
                try:
                    price = re.search('\$.* ', rent_block.text)
                    price = price.group(0)[:-2]
                except:
                    price = ""
                
                bedroom_info.append({
                    'bedroom_type': bedroom_type,
                    'price': price
                })
            
            rent_obj = {
                "Property_Name_Rent": clean_property_name,
                "City_Rent": city,
                "Type": "Apartment",
                "State_Rent": state,
                "Zip_code": zip_code,
                "Bedroom_Info": bedroom_info,
                
            }
            
            for item in bedroom_info:
                rent_obj[item["bedroom_type"]] = item["price"]
                

        
            all_property.append(rent_obj)
        except AttributeError as e:
            print(e)

# print(all_property)

In [58]:
house_data = pd.DataFrame(all_property)
house_data

,1 Bedroom – 2 Bedrooms,1 Bedroom – 3 Bedrooms,Studio,Studio – 3 Bedrooms,1 Bedroom,2 Bedrooms,3 Bedrooms,4 Bedrooms,Bedroom_Info,City_Rent,Property_Name_Rent,State_Rent,Studio,Type,Zip_code
0,NaN,NaN,NaN,NaN,"$2,356","$2,890",NaN,NaN,"[{'bedroom_type': 'Studio', 'price': '$2,030 '...",Los Angeles,Sakura Crossing,CA,"$2,030",Apartment,90012
1,NaN,NaN,NaN,NaN,"$1,849","$2,379","$3,424",NaN,"[{'bedroom_type': 'Studio', 'price': '$1,669 '...",Costa Mesa,27 Seventy Five Mesa Verde,CA,"$1,669",Apartment,92626
2,NaN,NaN,NaN,NaN,"$3,300",,,NaN,"[{'bedroom_type': 'Studio', 'price': ''}, {'be...",Berkeley,Parker,CA,,Apartment,94704
3,NaN,NaN,NaN,NaN,"$3,215","$3,800","$5,355",NaN,"[{'bedroom_type': 'Studio', 'price': ''}, {'be...",West Hollywood,Avalon West Hollywood,CA,,Apartment,90046
4,NaN,NaN,NaN,NaN,"$2,180","$3,070",,NaN,"[{'bedroom_type': '1 Bedroom', 'price': '$2,18...",Woodland Hills,Motif,CA,NaN,Apartment,91367
5,NaN,NaN,NaN,NaN,"$1,400","$1,805",,NaN,"[{'bedroom_type': 'Studio', 'price': ''}, {'be...",La Mesa,The Hills At Spring Street Apartments,CA,,Apartment,91941
6,NaN,NaN,NaN,NaN,"$2,655","$2,860","$4,080",NaN,"[{'bedroom_type': '1 Bedroom', 'price': '$2,65...",Fremont,Avalon Fremont,CA,NaN,Apartment,94538
7,NaN,NaN,NaN,NaN,"$2,639","$3,765",NaN,NaN,"[{'bedroom_type': 'Studio', 'price': '$2,405 '...",Mountain View,eaves Mt. View at Middlefield,CA,"$2,405",Apartment,94043
8,NaN,NaN,NaN,NaN,,"$1,425",,NaN,"[{'bedroom_type': '1 Bedroom', 'price': ''}, {...",Ontario,Mission Village,CA,NaN,Apartment,91762
9,NaN,NaN,NaN,NaN,"$2,165","$2,745",NaN,NaN,"[{'bedroom_type': 'Studio', 'price': '$1,950 '...",Irvine,Park Place Apartment Homes,CA,"$1,950",Apartment,92612


In [59]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Q2-2018_Apartment_Apartment_Data26.csv", index=False, header=True)